# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import joblib

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report

In [2]:
nltk.download(['punkt','stopwords','wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\allsi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\allsi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\allsi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('labeled_messages', engine)
pd.set_option('display.max_columns', None)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Define feature and target variables X and Y
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis=1)

In [5]:
X.isna().sum()

0

In [6]:
Y.isna().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    '''
    Description:
        Convert text to all lowercase characters,
        tokenize text, and stem and lemmatize text.
    
    Args:
        text - string - the message sent for processing
       
    Returns:
        filtered_sentence - list of strings - a list of the cleaned and stemmed message words
    '''
    
    # convert all text to lowercase characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    word_tokens = word_tokenize(text)
    
    # stem and lemmatize non-stop words to reduce redundancy
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed_sentence = [stemmer.stem(word) for word in word_tokens if word not in stop_words]
    filtered_sentence = [lemmatizer.lemmatize(word, pos='v') for word in stemmed_sentence]
    
    return filtered_sentence

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
model1 = RandomForestClassifier(random_state=42)

In [9]:
# Create pipeline, with help from https://datascience.stackexchange.com/questions/58377/using-a-linearsvc-for-multilabel-classification-with-multioutputclassifier-i
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(model1))
])

In [10]:
# pipeline.get_params().keys()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [12]:
# fit training data to the pipeline
pipeline.fit(X_train, y_train);

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
# get train target predictions
y_train_preds = pipeline.predict(X_train)

In [42]:
# get test target predictions
y_preds = pipeline.predict(X_test);

In [16]:
def get_scores(y_test, y_preds):
    '''
    Calculate and print f1, precision, and recall scores.
    
    Args:
    y_test - pandas dataframe - the test target values
    y_preds - pandas dataframe - the predicted target values
       
    Returns: nothing
    '''
    
    for i, column in enumerate(y_test):
        print(column)
        print(classification_report(y_test[column], y_preds[:, i]))

In [17]:
# print the train f1, precision, and recall scores for each output category
# of the dataset
get_scores(y_train, y_train_preds)

related
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.99      2463
         1.0       0.99      1.00      0.99      4644

    accuracy                           0.99      7107
   macro avg       0.99      0.99      0.99      7107
weighted avg       0.99      0.99      0.99      7107

request
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      4616
         1.0       1.00      0.98      0.99      2491

    accuracy                           0.99      7107
   macro avg       0.99      0.99      0.99      7107
weighted avg       0.99      0.99      0.99      7107

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7100
         1.0       1.00      1.00      1.00         7

    accuracy                           1.00      7107
   macro avg       1.00      1.00      1.00      7107
weighted avg       1.00      1.00      1.00      710

In [44]:
# print the test f1, precision, and recall scores for each output category
# of the dataset
get_scores(y_test, y_preds)

related
              precision    recall  f1-score   support

         0.0       0.37      0.06      0.10      1929
         1.0       0.76      0.97      0.85      5883

    accuracy                           0.74      7812
   macro avg       0.56      0.51      0.47      7812
weighted avg       0.66      0.74      0.66      7812

request
              precision    recall  f1-score   support

         0.0       0.83      0.99      0.90      6465
         1.0       0.39      0.04      0.08      1347

    accuracy                           0.82      7812
   macro avg       0.61      0.52      0.49      7812
weighted avg       0.76      0.82      0.76      7812

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7773
         1.0       0.00      0.00      0.00        39

    accuracy                           1.00      7812
   macro avg       0.50      0.50      0.50      7812
weighted avg       0.99      1.00      0.99      781

C:\Users\allsi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      6936
         1.0       0.12      0.00      0.01       876

    accuracy                           0.89      7812
   macro avg       0.51      0.50      0.47      7812
weighted avg       0.80      0.89      0.83      7812

shelter
              precision    recall  f1-score   support

         0.0       0.91      0.99      0.95      7112
         1.0       0.16      0.01      0.02       700

    accuracy                           0.91      7812
   macro avg       0.54      0.50      0.49      7812
weighted avg       0.84      0.91      0.87      7812

clothing
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      7688
         1.0       0.00      0.00      0.00       124

    accuracy                           0.98      7812
   macro avg       0.49      0.50      0.50      7812
weighted avg       0.97      0.98      0.98      7812

mo

### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
def improve_model(X, Y, model, params):
    '''
    Description:
        Improves a machine learning model using GridSearchCV to find the
        optimal hyperparameter for a given model.
    
    Args:
        X - pandas dataframe - the features values
        Y - pandas dataframe - the target values
        model - classifier - the machine learning classifier to be tuned
        params - dictionar - the hyperparameters to be used for model tuning
       
    Returns:
        y_test - pandas dataframe - the target values used for testing
        y_preds - pandas dataframe - the predicted target values
        cv.best_estimator_ - classifier - the best tuned classifier
    '''
    clf = model
    
    # with help from https://datascience.stackexchange.com/questions/58377
    # /using-a-linearsvc-for-multilabel-classification-with-multioutputclassifier-i
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(clf))
    ])
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

    pipeline.fit(X_train, y_train);
    
    parameters = params
    
    cv = GridSearchCV(pipeline, parameters)
    
    cv = cv.fit(X_train, y_train)
    
    y_train_preds = cv.predict(X_train)
    y_test_preds = cv.predict(X_test)
    
    return y_test, y_train_preds, y_test_preds, cv.best_estimator_
    

In [24]:
# set hyperparameters for RandomForestClassifier
parameters1 = {'clf__estimator__max_depth': list(range(20, 100, 20)),
               'clf__estimator__min_samples_split': [2, 3, 4],
              'clf__estimator__min_samples_leaf': [1, 2, 3]
              }

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
# improve RandomForestClassifier model
y_test, y_train_preds, y_test_preds, rf_model = improve_model(X, Y, model1, parameters1)

In [27]:
# print train scores for improved RandomForestClassifier model
get_scores(y_train, y_train_preds)

related
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      2463
         1.0       0.65      1.00      0.79      4644

    accuracy                           0.65      7107
   macro avg       0.33      0.50      0.40      7107
weighted avg       0.43      0.65      0.52      7107

request
              precision    recall  f1-score   support

         0.0       0.65      1.00      0.79      4616
         1.0       1.00      0.00      0.01      2491

    accuracy                           0.65      7107
   macro avg       0.83      0.50      0.40      7107
weighted avg       0.77      0.65      0.51      7107

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7100
         1.0       0.00      0.00      0.00         7

    accuracy                           1.00      7107
   macro avg       0.50      0.50      0.50      7107
weighted avg       1.00      1.00      1.00      710

C:\Users\allsi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6891
         1.0       0.00      0.00      0.00       216

    accuracy                           0.97      7107
   macro avg       0.48      0.50      0.49      7107
weighted avg       0.94      0.97      0.95      7107

transport
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      6972
         1.0       0.00      0.00      0.00       135

    accuracy                           0.98      7107
   macro avg       0.49      0.50      0.50      7107
weighted avg       0.96      0.98      0.97      7107

buildings
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      6853
         1.0       0.00      0.00      0.00       254

    accuracy                           0.96      7107
   macro avg       0.48      0.50      0.49      7107
weighted avg       0.93      0.96      0.95      7107


In [28]:
# print test scores for improved RandomForestClassifier model
get_scores(y_test, y_test_preds)

related
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1032
         1.0       0.66      1.00      0.80      2014

    accuracy                           0.66      3046
   macro avg       0.33      0.50      0.40      3046
weighted avg       0.44      0.66      0.53      3046

request
              precision    recall  f1-score   support

         0.0       0.65      1.00      0.79      1975
         1.0       0.00      0.00      0.00      1071

    accuracy                           0.65      3046
   macro avg       0.32      0.50      0.39      3046
weighted avg       0.42      0.65      0.51      3046

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3043
         1.0       0.00      0.00      0.00         3

    accuracy                           1.00      3046
   macro avg       0.50      0.50      0.50      3046
weighted avg       1.00      1.00      1.00      304

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
# check whether target column contains only 1s and 0s
print(np.unique(Y))

[0. 1.]


In [30]:
# try AdaBoostClassifier
model2 = AdaBoostClassifier(random_state=42)

# hyperparameters for AdaBoostClassifier
parameters2 = {'vect__min_df': [1, 2, 3],
               'tfidf__smooth_idf': [True, False],
               'clf__estimator__n_estimators':list(range(10, 100, 10)),
               'clf__estimator__learning_rate': [0.001, 0.01, 0.1, 1.0]
               }

In [31]:
# improve AdaBoostClassifier model
y_test, y_train_preds, y_test_preds, abc_model = improve_model(X, Y, model2, parameters2)

In [33]:
# print train scores for improved RandomForestClassifier model
get_scores(y_train, y_train_preds)

related
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      2463
         1.0       0.65      1.00      0.79      4644

    accuracy                           0.65      7107
   macro avg       0.33      0.50      0.40      7107
weighted avg       0.43      0.65      0.52      7107

request
              precision    recall  f1-score   support

         0.0       0.65      1.00      0.79      4616
         1.0       0.00      0.00      0.00      2491

    accuracy                           0.65      7107
   macro avg       0.32      0.50      0.39      7107
weighted avg       0.42      0.65      0.51      7107

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7100
         1.0       0.00      0.00      0.00         7

    accuracy                           1.00      7107
   macro avg       0.50      0.50      0.50      7107
weighted avg       1.00      1.00      1.00      710

C:\Users\allsi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.86      1.00      0.92      6096
         1.0       0.00      0.00      0.00      1011

    accuracy                           0.86      7107
   macro avg       0.43      0.50      0.46      7107
weighted avg       0.74      0.86      0.79      7107

infrastructure_related
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6891
         1.0       0.00      0.00      0.00       216

    accuracy                           0.97      7107
   macro avg       0.48      0.50      0.49      7107
weighted avg       0.94      0.97      0.95      7107

transport
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      6972
         1.0       0.00      0.00      0.00       135

    accuracy                           0.98      7107
   macro avg       0.49      0.50      0.50      7107
weighted avg       0.96      0.98      0.

In [34]:
# print test scores for improved AdaBoostClassifier model
get_scores(y_test, y_test_preds)

related
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1032
         1.0       0.66      1.00      0.80      2014

    accuracy                           0.66      3046
   macro avg       0.33      0.50      0.40      3046
weighted avg       0.44      0.66      0.53      3046

request
              precision    recall  f1-score   support

         0.0       0.65      1.00      0.79      1975
         1.0       0.00      0.00      0.00      1071

    accuracy                           0.65      3046
   macro avg       0.32      0.50      0.39      3046
weighted avg       0.42      0.65      0.51      3046

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3043
         1.0       0.00      0.00      0.00         3

    accuracy                           1.00      3046
   macro avg       0.50      0.50      0.50      3046
weighted avg       1.00      1.00      1.00      304

### 9. Export your model as a pickle file

In [35]:
joblib.dump(abc_model, 'disaster_model.pkl')

['disaster_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.